
# <span style="color:#006E7F">__Introduction to Oxford Nanopore Data Analysis__ <a class="anchor"></span>  

Created by C. Tranchant (DIADE-IRD), J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and A. Dereeper (PHIM-IRD) - May 2022  SouthGreen training 

Adapted by J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) - Novembre 2022
    
# <span style="color:#006E7F"> TP5. Structural Variants Detection using long reads  </center> </span>

</span>


### <span style="color:#006E7F">__0 - Preparing working environment__ <a class="anchor" id="work"></a></span>  
### <span style="color: #4CACBC;"> Create the working directory

In [ ]:
mkdir -p /home/jovyan/work/RESULTS/SV
cd /home/jovyan/work//RESULTS/
ls -l

### <span style="color: #4CACBC;"> Download two genomes</span>  

In [ ]:
wget https://itrop.ird.fr/sv-training/Assemblies.tar.gz
tar xvf Assemblies.tar.gz && rm Assemblies.tar.gz

In [ ]:
ls -R

## <span style="color:#006E7F">1. nucmer and syri <a class="anchor" id="SVsyri"></a></span>  

### <span style="color: #4CACBC;"> Initialization of two variables "genome"</span>  


In [ ]:
dir_genome="/home/jovyan/work/SV/"
reference_assembly=$dir_genome"Assemblies/A8_assembly.fasta"
query_assembly=$dir_genome"Assemblies/5417_assembly.fasta"

### <span style="color: #4CACBC;"> Create the working directory for SYRI analysis</span>  

In [ ]:
dir_SV=$dir_genome"SYRI/"
mkdir -p $dir_SV
cd $dir_SV

### <span style="color: #4CACBC;"> Aligning genomes using `Nucmer` <a class="anchor" id="nucmer"></a></span>  

#### Alignement of the two genomes

In [ ]:
nucmer --maxmatch $reference_assembly $query_assembly

In [ ]:
ls -lrt

#### Filtering nucmer results

Remove small and lower quality alignments


In [ ]:
delta-filter -m -i 90 -l 100 out.delta > out.filtered.delta

In [ ]:
ls -lrt

In [ ]:
show-coords -THrd out.filtered.delta > out.filtered.coords

In [ ]:
ls -lrt

### <span style="color: #4CACBC;"> SV calling using syri <a class="anchor" id="siri"></a></span>  

In [ ]:
syri -c out.filtered.coords -d out.filtered.delta -r $reference_assembly -q $query_assembly

In [ ]:
ls

In [ ]:
cat syri.summary

In [ ]:
head syri.vcf

### <span style="color: #4CACBC;"> Extracting all SNP from syri  <a class="anchor" id="siri"></a></span>  

In [ ]:
cat syri.out | grep SNP | cut -f 1,2 > a
cat syri.out | grep SNP | cut -f 4,5 > b
cat syri.out | grep SNP | cut -f 3 > c
awk '{ print $1+1 }' c > d
paste a d c > SNPs.bed
rm a b c d


In [ ]:
echo -e $reference_assembly'\t ref' > plotsr_pos.txt
echo -e $query_assembly'\t query' >> plotsr_pos.txt

In [ ]:
plotsr --sr syri.out --genomes plotsr_pos.txt -s 500 -o plotsr.pdf -H 8 -W 5